# Adaptive Learning Engine for Persuasion Training
This notebook implements an end-to-end adaptive learning pipeline that analyzes persuasion effectiveness in gaming recommendations. By combining large language models (LLMs) for feature extraction, contextual bandits for intelligent decision-making, and personalized coaching generation, we create a data-driven system that helps users improve their persuasion skills over time.

## Overview
1. extracts LLM-based features from every session transcript,
2. implements an adaptive policy to choose next-step focus,
3. produces adaptive coaching for the next session (focus skill + coaching card + scenario stub).
4. Results Export

## 1. Setup and Dependencies
First, let's install and import the required packages.

In [ ]:
# Install required packages
!pip install transformers torch numpy pandas plotly scikit-learn python-dotenv diskcache tqdm huggingface-hub

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from transformers import pipeline
import diskcache
import json
from pathlib import Path
from typing import List, Dict, Any
from tqdm.notebook import tqdm

## 2. Feature Extractor Implementation
We'll implement our PersuasionFeatureExtractor class that will compute 7 different features:
1. Objection Handling
2. Question Engagement
3. CTA Clarity
4. Empathy Score
5. Collaborative Language
6. Social Proof
7. Enthusiasm Level

In [ ]:
class PersuasionFeatureExtractor:
    def __init__(self, model_name: str = "facebook/bart-large-mnli"):
        """Initialize the feature extractor with the specified model."""
        self.classifier = pipeline("zero-shot-classification", model=model_name)
        self.cache = diskcache.Cache('./.cache/feature_extraction')
        
    def compute_objection_handling(self, text: str) -> float:
        """Measure how well objections are acknowledged and addressed."""
        cache_key = f"objection_handling_{hash(text)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        labels = [
            "acknowledges and addresses concerns",
            "ignores or dismisses concerns"
        ]
        result = self.classifier(text, labels, multi_label=False)
        score = result['scores'][0]
        self.cache[cache_key] = score
        return score
    
    def compute_question_ratio(self, turns: List[Dict[str, Any]]) -> float:
        """Calculate the ratio of turns containing questions."""
        question_count = sum(1 for turn in turns if '?' in turn['data'])
        return min(1.0, question_count / len(turns))
    
    def compute_cta_clarity(self, text: str) -> float:
        """Measure the clarity and specificity of the call to action."""
        cache_key = f"cta_clarity_{hash(text)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        labels = [
            "contains specific invitation with clear next steps",
            "vague or missing call to action"
        ]
        result = self.classifier(text, labels, multi_label=False)
        score = result['scores'][0]
        self.cache[cache_key] = score
        return score
    
    def compute_empathy_score(self, text: str) -> float:
        """Measure the level of empathy and emotional understanding."""
        cache_key = f"empathy_{hash(text)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        labels = [
            "shows understanding and emotional awareness",
            "lacks empathy or emotional connection"
        ]
        result = self.classifier(text, labels, multi_label=False)
        score = result['scores'][0]
        self.cache[cache_key] = score
        return score
    
    def compute_collaborative_score(self, text: str) -> float:
        """Measure the use of collaborative vs. pushy language."""
        cache_key = f"collaborative_{hash(text)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        labels = [
            "uses collaborative and inclusive language",
            "uses pushy or aggressive language"
        ]
        result = self.classifier(text, labels, multi_label=False)
        score = result['scores'][0]
        self.cache[cache_key] = score
        return score
    
    def compute_social_proof(self, text: str) -> float:
        """Measure references to community and shared experiences."""
        cache_key = f"social_proof_{hash(text)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        labels = [
            "references community or shared experiences",
            "lacks social context or community references"
        ]
        result = self.classifier(text, labels, multi_label=False)
        score = result['scores'][0]
        self.cache[cache_key] = score
        return score
    
    def compute_enthusiasm(self, text: str) -> float:
        """Measure genuine excitement and positive energy."""
        cache_key = f"enthusiasm_{hash(text)}"
        if cache_key in self.cache:
            return self.cache[cache_key]
            
        labels = [
            "shows genuine excitement and enthusiasm",
            "lacks energy or excitement"
        ]
        result = self.classifier(text, labels, multi_label=False)
        score = result['scores'][0]
        self.cache[cache_key] = score
        return score
    
    def extract_features(self, transcript: List[Dict[str, Any]]) -> Dict[str, float]:
        """Extract all features from a conversation transcript."""
        # Combine all user turns into one text for analysis
        user_text = " ".join(turn['data'] for turn in transcript if turn['type'] == 'user')
        
        features = {
            'objection_handling': self.compute_objection_handling(user_text),
            'question_ratio': self.compute_question_ratio(transcript),
            'cta_clarity': self.compute_cta_clarity(user_text),
            'empathy_score': self.compute_empathy_score(user_text),
            'collaborative_score': self.compute_collaborative_score(user_text),
            'social_proof': self.compute_social_proof(user_text),
            'enthusiasm': self.compute_enthusiasm(user_text)
        }
        
        return features

## 3. Data Loading and Processing
Now let's load our conversation data and process it using our feature extractor.

In [ ]:
file_path = "/kaggle/input/sanitized-sample/sanitized-sample.json"

# Load the conversation data
with open(file_path, 'r', encoding='utf-8-sig') as f:
    conversations = json.load(f)

print(f"Loaded {len(conversations)} conversations")

# Initialize our feature extractor
extractor = PersuasionFeatureExtractor()

# Process all conversations with a progress bar
results = []
for conv in tqdm(conversations, desc="Extracting features"):
    features = extractor.extract_features(conv['transcript'])
    features['conversation_id'] = conv['id']
    features['duration'] = (
        pd.to_datetime(conv['end_time']) -
        pd.to_datetime(conv['start_time'])
    ).total_seconds()
    results.append(features)

# Convert to DataFrame
features_df = pd.DataFrame(results)
features_df.head()

## 4. Feature Analysis and Visualization
Let's create some visualizations to understand our extracted features.

In [ ]:
# Feature distributions
fig = go.Figure()
for feature in features_df.columns:
    if feature not in ['conversation_id', 'duration']:
        fig.add_trace(go.Box(y=features_df[feature], name=feature))

fig.update_layout(
    title="Distribution of Persuasion Features",
    yaxis_title="Score",
    showlegend=False,
    height=500
)
fig.show()

In [ ]:
# Correlation heatmap
feature_cols = [col for col in features_df.columns if col not in ['conversation_id', 'duration']]
correlation_matrix = features_df[feature_cols].corr()

fig = go.Figure(data=go.Heatmap(
    z=correlation_matrix,
    x=feature_cols,
    y=feature_cols,
    colorscale='RdBu',
    zmin=-1,
    zmax=1
))

fig.update_layout(
    title="Feature Correlation Heatmap",
    height=600,
    width=800
)
fig.show()

## 5. Export Results
Finally, let's save our extracted features for future use.

In [ ]:
import os

# Create the directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Save to CSV
output_path = 'data/extracted_features.csv'
features_df.to_csv(output_path, index=False)
print(f"Features saved to {output_path}")

# Display summary statistics
print("\nFeature Statistics:")
print(features_df.describe())

## Next Steps
1. Use these features to train a model that predicts overall persuasion effectiveness
2. Implement an adaptive policy for choosing which skills to focus on
3. Generate personalized coaching recommendations based on the feature analysis

The extracted features provide a rich quantitative basis for analyzing persuasion effectiveness in gaming recommendations. The next notebook will focus on using these features to build an adaptive learning system.

## Part B: Adaptive Policy Implementation

### 1. Policy Implementation - LinUCB Contextual Bandit

In [ ]:
import numpy as np
from typing import Dict, List, Tuple, Optional
import pandas as pd
from dataclasses import dataclass

@dataclass
class ActionHistory:
    """Track action history for safety constraints."""
    action: str
    count: int = 1

class LinUCBPolicy:
    """
    Linear Upper Confidence Bound policy for skill focus selection.
    
    Action space: {clarity, active_listening, call_to_action, friendliness}
    Maps to rubric criteria:
    - clarity -> Clarity & Enthusiasm
    - active_listening -> Active Listening & Objection Handling
    - call_to_action -> Effective Call to Action
    - friendliness -> Friendliness & Respectful Tone
    """
    
    def __init__(self, n_features: int, alpha: float = 1.0, random_seed: int = 42):
        """
        Initialize LinUCB policy.
        
        Args:
            n_features: Dimension of context vector
            alpha: Exploration parameter (higher = more exploration)
            random_seed: For reproducibility
        """
        self.actions = ['clarity', 'active_listening', 'call_to_action', 'friendliness']
        self.action_to_rubric = {
            'clarity': 'Clarity & Enthusiasm',
            'active_listening': 'Active Listening & Objection Handling',
            'call_to_action': 'Effective Call to Action',
            'friendliness': 'Friendliness & Respectful Tone'
        }
        
        self.n_features = n_features
        self.alpha = alpha
        self.rng = np.random.RandomState(random_seed)
        
        # Initialize parameters for each action
        # A: design matrix (regularization + outer products)
        # b: response vector (weighted rewards)
        self.A = {action: np.eye(n_features) for action in self.actions}
        self.b = {action: np.zeros(n_features) for action in self.actions}
        
        # Action history for safety constraint
        self.action_history = []
        self.max_consecutive = 3
        
    def _get_context_vector(self, features: Dict[str, float], 
                           rubric_scores: Dict[str, float],
                           session_idx: int,
                           total_sessions: int) -> np.ndarray:
        """
        Build context vector from features and metadata.
        
        Context includes:
        - Current rubric scores (4 features)
        - LLM features (7 features)
        - Session progress (1 feature)
        - Improvement rate (1 feature, set to 0 for first session)
        """
        context = []
        
        # Rubric scores (normalized to [0,1])
        rubric_order = ['Clarity & Enthusiasm', 
                       'Active Listening & Objection Handling',
                       'Effective Call to Action', 
                       'Friendliness & Respectful Tone']
        context.extend([rubric_scores.get(r, 0) / 100.0 for r in rubric_order])
        
        # LLM features (already in [0,1])
        llm_features = ['objection_handling', 'question_ratio', 'cta_clarity',
                       'empathy_score', 'collaborative_score', 'social_proof', 
                       'enthusiasm']
        context.extend([features.get(f, 0) for f in llm_features])
        
        # Session progress
        progress = session_idx / max(1, total_sessions - 1)
        context.append(progress)
        
        # Improvement rate (placeholder for first session)
        context.append(0.0)
        
        return np.array(context)
    
    def _check_safety_constraint(self, action: str) -> bool:
        """Check if selecting this action violates safety constraint."""
        if len(self.action_history) < self.max_consecutive:
            return True
        
        recent_actions = self.action_history[-self.max_consecutive:]
        return not all(a == action for a in recent_actions)
    
    def select_action(self, context: np.ndarray, enforce_safety: bool = True) -> Tuple[str, Dict[str, float]]:
        """
        Select action using LinUCB algorithm with safety constraints.
        
        Returns:
            action: Selected action
            ucb_scores: UCB scores for all actions (for analysis)
        """
        ucb_scores = {}
        
        for action in self.actions:
            # Compute theta (parameter estimate)
            A_inv = np.linalg.inv(self.A[action])
            theta = A_inv @ self.b[action]
            
            # Compute UCB: predicted reward + uncertainty bonus
            prediction = theta @ context
            uncertainty = self.alpha * np.sqrt(context @ A_inv @ context)
            ucb = prediction + uncertainty
            
            # Apply safety constraint
            if enforce_safety and not self._check_safety_constraint(action):
                ucb -= 1000  # Large penalty to avoid selection
            
            ucb_scores[action] = ucb
        
        # Select action with highest UCB
        selected_action = max(ucb_scores, key=ucb_scores.get)
        self.action_history.append(selected_action)
        
        return selected_action, ucb_scores
    
    def update(self, action: str, context: np.ndarray, reward: float):
        """
        Update model parameters after observing reward.
        
        Args:
            action: Action taken
            context: Context vector when action was taken
            reward: Observed reward
        """
        self.A[action] += np.outer(context, context)
        self.b[action] += reward * context
    
    def compute_reward(self, current_rubric: Dict[str, float],
                      previous_rubric: Optional[Dict[str, float]],
                      action: str) -> float:
        """
        Compute reward: 0.6 * Δ(skill_focus) + 0.4 * Δ(overall)
        
        Args:
            current_rubric: Current session rubric scores
            previous_rubric: Previous session rubric scores (None for first session)
            action: Action that was taken
        """
        if previous_rubric is None:
            return 0.0  # No reward for first session
        
        # Map action to corresponding rubric criterion
        skill_name = self.action_to_rubric[action]
        
        # Compute improvements (normalized by 100)
        delta_skill = (current_rubric.get(skill_name, 0) - 
                      previous_rubric.get(skill_name, 0)) / 100.0
        delta_overall = (current_rubric.get('Overall', 0) - 
                        previous_rubric.get('Overall', 0)) / 100.0
        
        reward = 0.6 * delta_skill + 0.4 * delta_overall
        
        return reward

### 2. Load Rubric Scores

In [ ]:
# Load conversations with rubric scores
import json
file_path = "/kaggle/input/sanitized-sample/sanitized-sample.json"
with open(file_path, 'r', encoding='utf-8-sig') as f:
    conversations = json.load(f)

# Extract rubric scores for each conversation
rubric_data = []

for conv in conversations:
    assessment = conv.get("assessment_data", {})
    overall_score = assessment.get("overall", {}).get("score", None)
    criteria = assessment.get("criteria", {})

    rubric_scores = {
        "conversation_id": conv.get("id", None),
        "Overall": overall_score
    }

    # Add each criterion
    for name, details in criteria.items():
        rubric_scores[name] = details.get("score", None)

    rubric_data.append(rubric_scores)

rubric_df = pd.DataFrame(rubric_data)
print("Rubric Scores:")
print(rubric_df.head())

### 3. Merge Features with Rubric Scores

In [ ]:
print(globals().keys())

In [ ]:
# Merge LLM features with rubric scores
full_df = features_df.merge(rubric_df, on='conversation_id', how='inner')

# Sort by conversation start time (assuming conversation order = session order)
# For this demo, we'll use the original order as session sequence
full_df = full_df.reset_index(drop=True)
full_df['session_idx'] = range(len(full_df))

print(f"\nMerged dataset shape: {full_df.shape}")
print(full_df.columns.tolist())

### 4. Run LinUCB Policy (Offline Simulation)

In [ ]:
print(full_df.columns.tolist())

In [ ]:
# Initialize policy
n_features = 13  # 4 rubric + 7 LLM + 1 progress + 1 improvement_rate
policy = LinUCBPolicy(n_features=n_features, alpha=1.0, random_seed=42)

# Track results
policy_results = []

for idx in range(len(full_df)):
    current_row = full_df.iloc[idx]
    
    # Build context
    features = current_row[['objection_handling', 'question_ratio', 'cta_clarity',
                            'empathy_score', 'collaborative_score', 'social_proof', 
                            'enthusiasm']].to_dict()
    
    # Use correct lowercase column names
    rubric_scores = current_row[['clarity and enthusiasm in pitch', 
                                 'active listening and objection handling',
                                 'effective call to action', 
                                 'friendliness and respectful tone',
                                 'Overall']].to_dict()
    
    # Update improvement rate if not first session
    if idx > 0:
        prev_overall = full_df.iloc[idx-1]['Overall']
        improvement_rate = (rubric_scores['Overall'] - prev_overall) / 100.0
    else:
        improvement_rate = 0.0
    
    # Build context vector (depends on your LinUCBPolicy implementation)
    context = policy._get_context_vector(features, rubric_scores, idx, len(full_df))
    context[-1] = improvement_rate  # Update improvement rate
    
    # Select action
    action, ucb_scores = policy.select_action(context, enforce_safety=True)
    
    # Compute reward (using previous session for comparison)
    if idx > 0:
        prev_rubric = full_df.iloc[idx-1][['clarity and enthusiasm in pitch', 
                                           'active listening and objection handling',
                                           'effective call to action', 
                                           'friendliness and respectful tone',
                                           'Overall']].to_dict()
        reward = policy.compute_reward(rubric_scores, prev_rubric, action)
        
        # Update policy with reward
        policy.update(action, context, reward)
    else:
        reward = 0.0
    
    # Store results
    result = {
        'session_idx': idx,
        'conversation_id': current_row['conversation_id'],
        'selected_action': action,
        'reward': reward,
        'overall_score': rubric_scores['Overall'],
        'improvement_rate': improvement_rate
    }
    result.update({f'ucb_{k}': v for k, v in ucb_scores.items()})
    policy_results.append(result)

policy_df = pd.DataFrame(policy_results)
print("\nPolicy Results:")
print(policy_df.head(10))

### 5. Visualize Policy Performance

In [ ]:
# Action distribution
fig = px.histogram(policy_df, x='selected_action', 
                   title='Distribution of Selected Actions',
                   labels={'selected_action': 'Action', 'count': 'Frequency'})
fig.show()

# Reward over time
fig = px.line(policy_df, x='session_idx', y='reward',
              title='Reward Over Sessions',
              labels={'session_idx': 'Session', 'reward': 'Reward'})
fig.show()

# Overall score progression
fig = px.line(policy_df, x='session_idx', y='overall_score',
              title='Overall Score Progression',
              labels={'session_idx': 'Session', 'overall_score': 'Overall Score'})
fig.show()

# UCB scores over time (for first action as example)
ucb_cols = [col for col in policy_df.columns if col.startswith('ucb_')]
fig = go.Figure()
for col in ucb_cols:
    action_name = col.replace('ucb_', '')
    fig.add_trace(go.Scatter(x=policy_df['session_idx'], y=policy_df[col],
                            mode='lines', name=action_name))
fig.update_layout(title='UCB Scores Over Sessions',
                 xaxis_title='Session',
                 yaxis_title='UCB Score')
fig.show()

### 6. Safety Constraint Analysis

In [ ]:
# Check consecutive action streaks
def find_consecutive_streaks(actions):
    """Find maximum consecutive occurrences of each action."""
    max_streaks = {}
    for action in set(actions):
        max_streak = 0
        current_streak = 0
        for a in actions:
            if a == action:
                current_streak += 1
                max_streak = max(max_streak, current_streak)
            else:
                current_streak = 0
        max_streaks[action] = max_streak
    return max_streaks

streaks = find_consecutive_streaks(policy_df['selected_action'].tolist())
print("\nSafety Constraint Analysis:")
print(f"Maximum consecutive selections per action:")
for action, max_streak in streaks.items():
    status = "✓ SAFE" if max_streak <= 3 else "✗ VIOLATED"
    print(f"  {action}: {max_streak} {status}")

### 7. Save Results

In [ ]:
# Save policy results
policy_df.to_csv('data/policy_results.csv', index=False)
print("\nPolicy results saved to 'data/policy_results.csv'")

# Save action history
action_history_df = pd.DataFrame({
    'session_idx': range(len(policy.action_history)),
    'action': policy.action_history
})
action_history_df.to_csv('data/action_history.csv', index=False)
print("Action history saved to 'data/action_history.csv'")

# Part C: Adaptive Coaching Generation
# 
## This section implements personalized coaching cards and scenario generation
## based on the LinUCB policy decisions from Part B.

In [ ]:
import json
import random
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import pandas as pd

## 1. Coaching Card Generator

In [ ]:
@dataclass
class CoachingCard:
    """Structure for a coaching card."""
    session_idx: int
    focus_skill: str
    why_this_focus: str
    micro_exercises: List[str]
    word_count: int
    
    def to_dict(self) -> Dict:
        return {
            'session_idx': self.session_idx,
            'focus_skill': self.focus_skill,
            'why_this_focus': self.why_this_focus,
            'micro_exercises': self.micro_exercises,
            'word_count': self.word_count
        }
    
    def __str__(self) -> str:
        card = f"=== COACHING CARD: Session {self.session_idx + 1} ===\n\n"
        card += f"🎯 FOCUS: {self.focus_skill.replace('_', ' ').title()}\n\n"
        card += f"WHY THIS FOCUS:\n{self.why_this_focus}\n\n"
        card += "MICRO-EXERCISES:\n"
        for i, exercise in enumerate(self.micro_exercises, 1):
            card += f"{i}. {exercise}\n"
        card += f"\n[Word count: {self.word_count}]"
        return card


class CoachingCardGenerator:
    """Generate personalized coaching cards based on policy decisions."""
    
    def __init__(self):
        self.skill_descriptions = {
            'clarity': 'Clarity & Enthusiasm',
            'active_listening': 'Active Listening & Objection Handling',
            'call_to_action': 'Effective Call to Action',
            'friendliness': 'Friendliness & Respectful Tone'
        }
        
        # Exercise templates for each skill
        self.exercise_templates = {
            'clarity': [
                "Record yourself explaining {game} in 30 seconds. Count filler words ('um', 'like'). Aim for under 3.",
                "Write your top 3 selling points for {game}. Practice delivering each in one breath.",
                "Before your next call, create a 'hook sentence' that captures {game}'s unique appeal in under 15 words."
            ],
            'active_listening': [
                "During your next conversation, repeat back the prospect's concern in your own words before responding.",
                "Create a list of 5 common objections. For each, write a validating phrase (e.g., 'I hear that...').",
                "Practice the 3-second pause: After they finish speaking, count to 3 before you respond. Notice what you catch."
            ],
            'call_to_action': [
                "Write 3 versions of your closing invitation. Each must include a specific day/time option.",
                "Practice saying: 'Based on what you've shared, I think [specific session] would be perfect. Can you join us Thursday at 7 PM?'",
                "Role-play with a friend: Give them an objection, then practice pivoting to a concrete next step."
            ],
            'friendliness': [
                "Start your next call by finding one genuine commonality with the prospect within the first 2 minutes.",
                "Record yourself speaking. Listen back: Do you sound like you're smiling? Adjust your tone to sound warmer.",
                "Practice using their name 2-3 times naturally during the conversation. Notice how it changes the dynamic."
            ]
        }
        
        # Reason templates based on transcript signals
        self.reason_templates = {
            'clarity': {
                'low_enthusiasm': "Your transcript shows lower enthusiasm scores ({enthusiasm:.2f}). Injecting more genuine excitement helps prospects feel your passion for {game}.",
                'low_cta': "Your call-to-action clarity scored {cta_clarity:.2f}. Clearer invitations make it easier for prospects to say yes.",
                'general': "Your clarity metrics suggest room for sharper, more compelling messaging about {game}'s value."
            },
            'active_listening': {
                'low_objection': "Your objection handling score is {objection_handling:.2f}. Better acknowledgment of concerns builds trust faster.",
                'low_questions': "You asked questions in only {question_ratio:.0%} of turns. More questions = deeper understanding of their needs.",
                'general': "Your transcript shows opportunities to demonstrate deeper listening and validation of concerns."
            },
            'call_to_action': {
                'low_cta': "Your CTA clarity scored {cta_clarity:.2f}. Specific, actionable invitations convert better than vague suggestions.",
                'low_collaborative': "Your collaborative score is {collaborative_score:.2f}. Frame CTAs as 'next steps together' rather than pressure.",
                'general': "Your closing invitations could be more concrete and easier for prospects to act on."
            },
            'friendliness': {
                'low_empathy': "Your empathy score is {empathy_score:.2f}. Warmer connection early makes the whole conversation flow better.",
                'low_social': "Social proof score: {social_proof:.2f}. Mentioning community experiences makes invitations feel less intimidating.",
                'general': "Your tone could benefit from more warmth and personal connection to put prospects at ease."
            }
        }
    
    def _select_reason_template(self, focus_skill: str, features: Dict[str, float]) -> str:
        """Select appropriate reason template based on feature scores."""
        templates = self.reason_templates[focus_skill]
        
        if focus_skill == 'clarity':
            if features['enthusiasm'] < 0.5:
                return templates['low_enthusiasm']
            elif features['cta_clarity'] < 0.5:
                return templates['low_cta']
            return templates['general']
        
        elif focus_skill == 'active_listening':
            if features['objection_handling'] < 0.5:
                return templates['low_objection']
            elif features['question_ratio'] < 0.3:
                return templates['low_questions']
            return templates['general']
        
        elif focus_skill == 'call_to_action':
            if features['cta_clarity'] < 0.5:
                return templates['low_cta']
            elif features['collaborative_score'] < 0.5:
                return templates['low_collaborative']
            return templates['general']
        
        else:  # friendliness
            if features['empathy_score'] < 0.5:
                return templates['low_empathy']
            elif features['social_proof'] < 0.5:
                return templates['low_social']
            return templates['general']
    
    def generate_card(self, session_idx: int, focus_skill: str, 
                     features: Dict[str, float], 
                     game: str = "the game") -> CoachingCard:
        """Generate a complete coaching card."""
        
        # Generate "why this focus" (2-3 lines)
        reason_template = self._select_reason_template(focus_skill, features)
        why_text = reason_template.format(**features, game=game)
        
        # Select 3 exercises
        exercises = [ex.format(game=game) for ex in self.exercise_templates[focus_skill]]
        
        # Calculate word count
        full_text = why_text + " " + " ".join(exercises)
        word_count = len(full_text.split())
        
        # Ensure within 120-180 word limit
        if word_count > 180:
            # Truncate exercises if needed
            while word_count > 180 and len(exercises) > 1:
                exercises = exercises[:-1]
                full_text = why_text + " " + " ".join(exercises)
                word_count = len(full_text.split())
        
        return CoachingCard(
            session_idx=session_idx,
            focus_skill=self.skill_descriptions[focus_skill],
            why_this_focus=why_text,
            micro_exercises=exercises,
            word_count=word_count
        )

## 2. Scenario Stub Generator

In [ ]:
@dataclass
class ScenarioStub:
    """Structure for a practice scenario stub."""
    session_idx: int
    focus_skill: str
    difficulty: float
    persona: str
    opening_objection: str
    follow_ups: List[str]
    difficulty_toggles: Dict[str, str]
    
    def to_dict(self) -> Dict:
        return {
            'session_idx': self.session_idx,
            'focus_skill': self.focus_skill,
            'difficulty': self.difficulty,
            'persona': self.persona,
            'opening_objection': self.opening_objection,
            'follow_ups': self.follow_ups,
            'difficulty_toggles': self.difficulty_toggles
        }
    
    def __str__(self) -> str:
        stub = f"=== PRACTICE SCENARIO: Session {self.session_idx + 1} ===\n\n"
        stub += f"🎮 FOCUS SKILL: {self.focus_skill}\n"
        stub += f"📊 BASE DIFFICULTY: {self.difficulty}\n\n"
        stub += f"PERSONA: {self.persona}\n\n"
        stub += f"OPENING:\n\"{self.opening_objection}\"\n\n"
        stub += "FOLLOW-UP CHALLENGES:\n"
        for i, followup in enumerate(self.follow_ups, 1):
            stub += f"{i}. \"{followup}\"\n"
        stub += f"\n🔧 DIFFICULTY TOGGLES (0.55):\n"
        for toggle, description in self.difficulty_toggles.items():
            stub += f"  [{toggle}] {description}\n"
        return stub


class ScenarioGenerator:
    """Generate practice scenarios tailored to focus skills."""
    
    def __init__(self, random_seed: int = 42):
        self.rng = random.Random(random_seed)
        
        # Persona templates
        self.personas = {
            'clarity': [
                "Busy parent, 35, skeptical of gaming, needs clear value proposition",
                "College student, 20, distracted, needs compelling hook to stay engaged",
                "Professional, 42, time-conscious, wants efficiency and clear benefits"
            ],
            'active_listening': [
                "Former gamer, 28, burned out, has specific past negative experiences",
                "Anxious newcomer, 25, worried about skill level and fitting in",
                "Critical thinker, 38, asks probing questions to test your knowledge"
            ],
            'call_to_action': [
                "Interested but non-committal, 30, needs gentle push to concrete action",
                "Over-thinker, 33, gets stuck in analysis paralysis",
                "Schedule-juggler, 29, needs very specific timing options"
            ],
            'friendliness': [
                "Shy introvert, 24, needs warmth to open up about interests",
                "Defensive skeptic, 36, testing whether you're 'just selling'",
                "Lonely player, 31, craves community but fears rejection"
            ]
        }
        
        # Opening objections tailored to each skill
        self.opening_objections = {
            'clarity': [
                "I've heard of this game, but I'm not really sure what makes it special. Can you explain?",
                "Okay, so... what exactly would I be doing? I'm kind of confused about the gameplay.",
                "Why should I play this instead of [popular alternative]? What's the difference?"
            ],
            'active_listening': [
                "I tried a game like this before and the community was toxic. How is this different?",
                "I'm really bad at these types of games. I don't want to embarrass myself.",
                "I'm interested, but I have some concerns about the time commitment and learning curve."
            ],
            'call_to_action': [
                "This sounds interesting, but I need to think about it. Can I get back to you?",
                "I might be interested... maybe sometime next month when things calm down?",
                "I'm not sure I'm ready to commit to a session yet. What if I just lurk first?"
            ],
            'friendliness': [
                "I don't know anyone who plays this. Would I just be joining random strangers?",
                "Are you getting paid to recruit people? This feels like a sales pitch.",
                "I'm not really a 'joiner' type. I prefer playing solo."
            ]
        }
        
        # Follow-ups that stress the specific skill
        self.follow_ups = {
            'clarity': [
                "You're giving me a lot of information... can you just give me the main reason to try it?",
                "I still don't get it. What would my first hour actually look like?"
            ],
            'active_listening': [
                "It sounds like you're just reading from a script. Did you hear what I just said about my schedule?",
                "You keep talking about features, but I told you my main concern is [X]. Can you address that?"
            ],
            'call_to_action': [
                "That's a lot to consider. What exactly are you asking me to do right now?",
                "I hear you, but 'sometime soon' is too vague. When specifically works for me?"
            ],
            'friendliness': [
                "You seem nice, but I'm still not convinced this is for someone like me.",
                "How do I know the other players will be welcoming? I've had bad experiences."
            ]
        }
        
        # Difficulty toggles for 0.55 difficulty
        self.difficulty_toggles = {
            'clarity': {
                'Time Pressure': 'They interrupt after 90 seconds: "Can you wrap this up? I have another call."',
                'Stronger Skepticism': 'They push back twice as hard: "My friend said this game is overhyped. Prove them wrong."'
            },
            'active_listening': {
                'Multiple Concerns': 'They layer 3 objections at once instead of 1.',
                'Emotional Intensity': 'They sound frustrated/anxious; mishandling tone ends the call.'
            },
            'call_to_action': {
                'Time Pressure': 'You have only 2 minutes to secure commitment before they "have to go."',
                'Analysis Paralysis': 'They ask for 3+ more details before committing; you must redirect firmly but kindly.'
            },
            'friendliness': {
                'Cold Opening': 'They start defensive: "Make this quick. I don\'t have much time."',
                'Trust Test': 'They explicitly test if you\'re authentic: "Are you reading from a script?"'
            }
        }
    
    def generate_scenario(self, session_idx: int, focus_skill: str, 
                         difficulty: float = 0.45) -> ScenarioStub:
        """Generate a practice scenario stub."""
        
        # Select random persona and objections
        persona = self.rng.choice(self.personas[focus_skill])
        opening = self.rng.choice(self.opening_objections[focus_skill])
        followups = self.follow_ups[focus_skill].copy()
        
        return ScenarioStub(
            session_idx=session_idx,
            focus_skill=focus_skill.replace('_', ' ').title(),
            difficulty=difficulty,
            persona=persona,
            opening_objection=opening,
            follow_ups=followups,
            difficulty_toggles=self.difficulty_toggles[focus_skill]
        )

## 3. Integration with Policy Results

In [ ]:
def generate_coaching_pipeline(policy_df: pd.DataFrame, 
                               features_df: pd.DataFrame,
                               output_dir: str = 'data') -> Tuple[List[CoachingCard], List[ScenarioStub]]:
    """
    Generate coaching cards and scenarios for all sessions.
    
    Args:
        policy_df: DataFrame with policy results (from Part B)
        features_df: DataFrame with LLM features (from Part A)
        output_dir: Directory to save outputs
        
    Returns:
        Tuple of (coaching_cards, scenario_stubs)
    """
    card_gen = CoachingCardGenerator()
    scenario_gen = ScenarioGenerator()
    
    coaching_cards = []
    scenario_stubs = []
    
    # Merge dataframes
    merged = policy_df.merge(
        features_df[['conversation_id', 'objection_handling', 'question_ratio', 
                     'cta_clarity', 'empathy_score', 'collaborative_score', 
                     'social_proof', 'enthusiasm']], 
        on='conversation_id'
    )
    
    for idx, row in merged.iterrows():
        # Get focus skill (map back to action name)
        focus_skill = row['selected_action']
        
        # Extract features
        features = {
            'objection_handling': row['objection_handling'],
            'question_ratio': row['question_ratio'],
            'cta_clarity': row['cta_clarity'],
            'empathy_score': row['empathy_score'],
            'collaborative_score': row['collaborative_score'],
            'social_proof': row['social_proof'],
            'enthusiasm': row['enthusiasm']
        }
        
        # Generate coaching card
        card = card_gen.generate_card(
            session_idx=idx,
            focus_skill=focus_skill,
            features=features,
            game="our gaming community"
        )
        coaching_cards.append(card)
        
        # Generate scenario stub
        scenario = scenario_gen.generate_scenario(
            session_idx=idx,
            focus_skill=focus_skill,
            difficulty=0.45
        )
        scenario_stubs.append(scenario)
    
    return coaching_cards, scenario_stubs

## 4. Run Coaching Generation

In [ ]:
# Generate coaching materials
coaching_cards, scenario_stubs = generate_coaching_pipeline(
    policy_df=policy_df,
    features_df=features_df,
    output_dir='data'
)

print(f"Generated {len(coaching_cards)} coaching cards and {len(scenario_stubs)} scenario stubs")

# Display first 3 examples
print("\n" + "="*70)
print("EXAMPLE COACHING OUTPUTS")
print("="*70)

for i in range(min(3, len(coaching_cards))):
    print(f"\n{coaching_cards[i]}")
    print(f"\n{scenario_stubs[i]}")
    print("\n" + "-"*70)

## 5. Save Coaching Materials

In [ ]:
import os

# Create output directory
os.makedirs('data/coaching_materials', exist_ok=True)

# Save coaching cards
cards_data = [card.to_dict() for card in coaching_cards]
with open('data/coaching_materials/coaching_cards.json', 'w') as f:
    json.dump(cards_data, f, indent=2)

# Save as readable text
with open('data/coaching_materials/coaching_cards.txt', 'w') as f:
    for card in coaching_cards:
        f.write(str(card))
        f.write("\n\n" + "="*70 + "\n\n")

# Save scenario stubs
scenarios_data = [scenario.to_dict() for scenario in scenario_stubs]
with open('data/coaching_materials/scenario_stubs.json', 'w') as f:
    json.dump(scenarios_data, f, indent=2)

# Save as readable text
with open('data/coaching_materials/scenario_stubs.txt', 'w') as f:
    for scenario in scenario_stubs:
        f.write(str(scenario))
        f.write("\n\n" + "="*70 + "\n\n")

print("✅ Coaching materials saved to 'data/coaching_materials/'")
print("   - coaching_cards.json & .txt")
print("   - scenario_stubs.json & .txt")

## 6. Coaching Effectiveness Analysis

In [ ]:
# Analyze coaching card statistics
card_stats = pd.DataFrame([
    {
        'session_idx': card.session_idx,
        'focus_skill': card.focus_skill,
        'word_count': card.word_count,
        'num_exercises': len(card.micro_exercises)
    }
    for card in coaching_cards
])

print("\nCoaching Card Statistics:")
print(card_stats.describe())

# Focus skill distribution
skill_dist = card_stats['focus_skill'].value_counts()
print("\nFocus Skill Distribution:")
print(skill_dist)

# Visualize
import plotly.express as px
fig = px.bar(skill_dist, 
             title='Distribution of Focus Skills in Coaching Cards',
             labels={'value': 'Count', 'index': 'Focus Skill'})
fig.show()

# Word count distribution
fig = px.histogram(card_stats, x='word_count',
                   title='Coaching Card Word Count Distribution',
                   labels={'word_count': 'Word Count'},
                   nbins=20)
fig.add_vline(x=120, line_dash="dash", line_color="green", 
              annotation_text="Min (120)")
fig.add_vline(x=180, line_dash="dash", line_color="red",
              annotation_text="Max (180)")
fig.show()

## 7. Export Complete Training Pipeline

In [ ]:
# Create a complete training session export
def create_training_session_export(session_idx: int) -> Dict:
    """Create a complete training session package."""
    return {
        'session_number': session_idx + 1,
        'policy_decision': {
            'selected_action': policy_df.iloc[session_idx]['selected_action'],
            'reward': policy_df.iloc[session_idx]['reward'],
            'ucb_scores': {
                action: policy_df.iloc[session_idx][f'ucb_{action}']
                for action in ['clarity', 'active_listening', 'call_to_action', 'friendliness']
            }
        },
        'coaching_card': coaching_cards[session_idx].to_dict(),
        'practice_scenario': scenario_stubs[session_idx].to_dict(),
        'current_performance': {
            'overall_score': policy_df.iloc[session_idx]['overall_score'],
            'improvement_rate': policy_df.iloc[session_idx]['improvement_rate']
        }
    }

# Export all training sessions
training_pipeline = [
    create_training_session_export(i) 
    for i in range(len(coaching_cards))
]

with open('data/coaching_materials/complete_training_pipeline.json', 'w') as f:
    json.dump(training_pipeline, f, indent=2)

print("✅ Complete training pipeline exported to 'data/coaching_materials/complete_training_pipeline.json'")
print(f"\n📊 Pipeline Summary:")
print(f"   Total Sessions: {len(training_pipeline)}")
print(f"   Average Word Count: {card_stats['word_count'].mean():.1f}")
print(f"   Word Count Range: {card_stats['word_count'].min()}-{card_stats['word_count'].max()}")
print(f"   Skills Covered: {card_stats['focus_skill'].nunique()}")

## Part D: Lightweight Evaluation
# 
### This section evaluates:
### 1. Feature usefulness: Do LLM features improve prediction of score improvements?
### 2. Policy performance: How does LinUCB compare to baseline policies?
### 3. Alignment sanity: Does the policy focus on actual weaknesses?

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, roc_auc_score, accuracy_score
from sklearn.model_selection import LeaveOneOut
from typing import Dict, List, Tuple
import json

## 1. Feature Usefulness Evaluation
# 
### We'll use Leave-One-Step-Out (LOSO) cross-validation to predict:
### - Regression: Δoverall (continuous improvement score)
### - Classification: Positive Δoverall (yes/no)

In [ ]:
class FeatureUsefulnessEvaluator:
    """Evaluate predictive power of LLM features using LOSO."""
    
    def __init__(self, data_df: pd.DataFrame):
        """
        Initialize evaluator.
        
        Args:
            data_df: Merged dataframe with features, rubric scores, and metadata
        """
        self.data = data_df.copy()
        self._prepare_data()
    
    def _prepare_data(self):
        """Prepare features and targets for evaluation."""
        # Calculate Δoverall (t→t+1)
        self.data['delta_overall'] = self.data['Overall'].diff().shift(-1)
        self.data['positive_delta'] = (self.data['delta_overall'] > 0).astype(int)
        
        # Remove last row (no next step)
        self.data = self.data[:-1].copy()
        
        # Define feature sets
        self.rubric_features = [
            'clarity and enthusiasm in pitch',
            'active listening and objection handling',
            'effective call to action',
            'friendliness and respectful tone'
        ]
        
        self.llm_features = [
            'objection_handling', 'question_ratio', 'cta_clarity',
            'empathy_score', 'collaborative_score', 'social_proof', 
            'enthusiasm'
        ]
        
        # Add metadata features
        self.metadata_features = ['duration']  # talk ratio could be added if available
        
        # Baseline: rubric + metadata
        self.baseline_features = self.rubric_features + self.metadata_features
        
        # Full: baseline + LLM features
        self.full_features = self.baseline_features + self.llm_features
    
    def _loso_regression(self, features: List[str]) -> Dict:
        """Perform LOSO regression to predict Δoverall."""
        X = self.data[features].values
        y = self.data['delta_overall'].values
        
        loo = LeaveOneOut()
        predictions = np.zeros(len(y))
        
        for train_idx, test_idx in loo.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train = y[train_idx]
            
            model = LinearRegression()
            model.fit(X_train, y_train)
            predictions[test_idx] = model.predict(X_test)
        
        r2 = r2_score(y, predictions)
        mae = np.mean(np.abs(y - predictions))
        
        return {
            'r2': r2,
            'mae': mae,
            'predictions': predictions,
            'actuals': y
        }
    
    def _loso_classification(self, features: List[str]) -> Dict:
        """Perform LOSO classification to predict positive Δoverall."""
        X = self.data[features].values
        y = self.data['positive_delta'].values
        
        loo = LeaveOneOut()
        predictions = np.zeros(len(y))
        probabilities = np.zeros(len(y))
        
        for train_idx, test_idx in loo.split(X):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train = y[train_idx]
            
            model = LogisticRegression(max_iter=1000, random_state=42)
            model.fit(X_train, y_train)
            predictions[test_idx] = model.predict(X_test)
            probabilities[test_idx] = model.predict_proba(X_test)[0, 1]
        
        accuracy = accuracy_score(y, predictions)
        auc = roc_auc_score(y, probabilities)
        
        return {
            'accuracy': accuracy,
            'auc': auc,
            'predictions': predictions,
            'probabilities': probabilities,
            'actuals': y
        }
    
    def run_ablation_study(self) -> pd.DataFrame:
        """Run complete ablation study comparing feature sets."""
        results = []
        
        # 1. Baseline (rubric + metadata)
        print("Evaluating baseline features...")
        reg_baseline = self._loso_regression(self.baseline_features)
        clf_baseline = self._loso_classification(self.baseline_features)
        
        results.append({
            'Feature Set': 'Baseline (Rubric + Metadata)',
            'R² (Regression)': reg_baseline['r2'],
            'MAE (Regression)': reg_baseline['mae'],
            'Accuracy (Classification)': clf_baseline['accuracy'],
            'AUC (Classification)': clf_baseline['auc'],
            'Num Features': len(self.baseline_features)
        })
        
        # 2. Full (baseline + LLM)
        print("Evaluating full feature set...")
        reg_full = self._loso_regression(self.full_features)
        clf_full = self._loso_classification(self.full_features)
        
        results.append({
            'Feature Set': 'Full (Baseline + LLM)',
            'R² (Regression)': reg_full['r2'],
            'MAE (Regression)': reg_full['mae'],
            'Accuracy (Classification)': clf_full['accuracy'],
            'AUC (Classification)': clf_full['auc'],
            'Num Features': len(self.full_features)
        })
        
        # 3. LLM only
        print("Evaluating LLM features only...")
        reg_llm = self._loso_regression(self.llm_features)
        clf_llm = self._loso_classification(self.llm_features)
        
        results.append({
            'Feature Set': 'LLM Only',
            'R² (Regression)': reg_llm['r2'],
            'MAE (Regression)': reg_llm['mae'],
            'Accuracy (Classification)': clf_llm['accuracy'],
            'AUC (Classification)': clf_llm['auc'],
            'Num Features': len(self.llm_features)
        })
        
        # Calculate improvements
        results_df = pd.DataFrame(results)
        
        # Store detailed results for plotting
        self.regression_results = {
            'baseline': reg_baseline,
            'full': reg_full,
            'llm_only': reg_llm
        }
        self.classification_results = {
            'baseline': clf_baseline,
            'full': clf_full,
            'llm_only': clf_llm
        }
        
        return results_df

## 2. Run Feature Usefulness Evaluation

In [ ]:
# Initialize evaluator with merged data
evaluator = FeatureUsefulnessEvaluator(full_df)

# Run ablation study
ablation_results = evaluator.run_ablation_study()

print("\n" + "="*80)
print("ABLATION STUDY RESULTS")
print("="*80)
print(ablation_results.to_string(index=False))

# Calculate improvement from baseline to full
baseline_r2 = ablation_results.iloc[0]['R² (Regression)']
full_r2 = ablation_results.iloc[1]['R² (Regression)']
r2_improvement = ((full_r2 - baseline_r2) / abs(baseline_r2)) * 100 if baseline_r2 != 0 else float('inf')

baseline_auc = ablation_results.iloc[0]['AUC (Classification)']
full_auc = ablation_results.iloc[1]['AUC (Classification)']
auc_improvement = ((full_auc - baseline_auc) / baseline_auc) * 100

print(f"\n📊 LLM Feature Impact:")
print(f"   R² Improvement: {r2_improvement:+.1f}%")
print(f"   AUC Improvement: {auc_improvement:+.1f}%")

## 3. Visualize Prediction Performance

In [ ]:
# Regression: Actual vs Predicted
fig = go.Figure()

for name, results in evaluator.regression_results.items():
    fig.add_trace(go.Scatter(
        x=results['actuals'],
        y=results['predictions'],
        mode='markers',
        name=name.replace('_', ' ').title(),
        opacity=0.6
    ))

# Add perfect prediction line
min_val = min(evaluator.regression_results['baseline']['actuals'].min(),
              evaluator.regression_results['full']['actuals'].min())
max_val = max(evaluator.regression_results['baseline']['actuals'].max(),
              evaluator.regression_results['full']['actuals'].max())
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    name='Perfect Prediction',
    line=dict(dash='dash', color='gray')
))

fig.update_layout(
    title='Regression: Actual vs Predicted Δoverall (LOSO)',
    xaxis_title='Actual Δoverall',
    yaxis_title='Predicted Δoverall',
    height=500
)
fig.show()

# Classification: ROC-style comparison
from sklearn.metrics import roc_curve

fig = go.Figure()

for name, results in evaluator.classification_results.items():
    fpr, tpr, _ = roc_curve(results['actuals'], results['probabilities'])
    auc = results['auc']
    
    fig.add_trace(go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name=f"{name.replace('_', ' ').title()} (AUC={auc:.3f})"
    ))

# Add random classifier line
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode='lines',
    name='Random Classifier',
    line=dict(dash='dash', color='gray')
))

fig.update_layout(
    title='ROC Curves: Predicting Positive Δoverall (LOSO)',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=500
)
fig.show()

## 4. Policy Performance Evaluation

In [ ]:
class PolicyEvaluator:
    """Compare LinUCB policy against baseline policies."""
    
    def __init__(self, data_df: pd.DataFrame, policy_results_df: pd.DataFrame):
        """
        Initialize policy evaluator.
        
        Args:
            data_df: Merged dataframe with features and rubric scores
            policy_results_df: Results from LinUCB policy
        """
        self.data = data_df.copy()
        self.linucb_results = policy_results_df.copy()
        
        # Map rubric columns to actions
        self.rubric_to_action = {
            'clarity and enthusiasm in pitch': 'clarity',
            'active listening and objection handling': 'active_listening',
            'effective call to action': 'call_to_action',
            'friendliness and respectful tone': 'friendliness'
        }
    
    def _weakest_skill_first_policy(self) -> pd.DataFrame:
        """Simulate weakest-skill-first baseline policy."""
        results = []
        
        for idx in range(len(self.data) - 1):  # Exclude last row (no next step)
            current_row = self.data.iloc[idx]
            next_row = self.data.iloc[idx + 1]
            
            # Find weakest rubric score
            rubric_scores = {
                skill: current_row[rubric]
                for rubric, skill in self.rubric_to_action.items()
            }
            weakest_skill = min(rubric_scores, key=rubric_scores.get)
            
            # Calculate reward (same as LinUCB)
            current_rubric = {
                rubric: current_row[rubric]
                for rubric in self.rubric_to_action.keys()
            }
            current_rubric['Overall'] = current_row['Overall']
            
            next_rubric = {
                rubric: next_row[rubric]
                for rubric in self.rubric_to_action.keys()
            }
            next_rubric['Overall'] = next_row['Overall']
            
            # Map action to rubric
            action_to_rubric = {v: k for k, v in self.rubric_to_action.items()}
            skill_rubric = action_to_rubric[weakest_skill]
            
            delta_skill = (next_rubric[skill_rubric] - current_rubric[skill_rubric]) / 100.0
            delta_overall = (next_rubric['Overall'] - current_rubric['Overall']) / 100.0
            reward = 0.6 * delta_skill + 0.4 * delta_overall
            
            # Calculate Δoverall
            delta_overall_raw = next_row['Overall'] - current_row['Overall']
            
            results.append({
                'session_idx': idx,
                'selected_action': weakest_skill,
                'reward': reward,
                'delta_overall': delta_overall_raw,
                'positive_delta': delta_overall_raw > 0
            })
        
        return pd.DataFrame(results)
    
    def _random_policy(self, seed: int = 42) -> pd.DataFrame:
        """Simulate random action selection policy."""
        rng = np.random.RandomState(seed)
        actions = list(self.rubric_to_action.values())
        
        results = []
        
        for idx in range(len(self.data) - 1):
            current_row = self.data.iloc[idx]
            next_row = self.data.iloc[idx + 1]
            
            # Random action
            selected_action = rng.choice(actions)
            
            # Calculate reward
            current_rubric = {
                rubric: current_row[rubric]
                for rubric in self.rubric_to_action.keys()
            }
            current_rubric['Overall'] = current_row['Overall']
            
            next_rubric = {
                rubric: next_row[rubric]
                for rubric in self.rubric_to_action.keys()
            }
            next_rubric['Overall'] = next_row['Overall']
            
            action_to_rubric = {v: k for k, v in self.rubric_to_action.items()}
            skill_rubric = action_to_rubric[selected_action]
            
            delta_skill = (next_rubric[skill_rubric] - current_rubric[skill_rubric]) / 100.0
            delta_overall = (next_rubric['Overall'] - current_rubric['Overall']) / 100.0
            reward = 0.6 * delta_skill + 0.4 * delta_overall
            
            delta_overall_raw = next_row['Overall'] - current_row['Overall']
            
            results.append({
                'session_idx': idx,
                'selected_action': selected_action,
                'reward': reward,
                'delta_overall': delta_overall_raw,
                'positive_delta': delta_overall_raw > 0
            })
        
        return pd.DataFrame(results)
    
    def compare_policies(self) -> pd.DataFrame:
        """Compare all policies."""
        # Generate baseline policies
        weakest_results = self._weakest_skill_first_policy()
        random_results = self._random_policy()
        
        # Add delta_overall to LinUCB results if not present
        linucb_with_delta = self.linucb_results.copy()
        if 'delta_overall' not in linucb_with_delta.columns:
            # Calculate delta_overall from the data
            delta_overall_list = []
            positive_delta_list = []
            
            for idx in range(len(linucb_with_delta)):
                session_idx = linucb_with_delta.iloc[idx]['session_idx']
                if session_idx < len(self.data) - 1:
                    current_overall = self.data.iloc[session_idx]['Overall']
                    next_overall = self.data.iloc[session_idx + 1]['Overall']
                    delta = next_overall - current_overall
                    delta_overall_list.append(delta)
                    positive_delta_list.append(delta > 0)
                else:
                    delta_overall_list.append(np.nan)
                    positive_delta_list.append(False)
            
            linucb_with_delta['delta_overall'] = delta_overall_list
            linucb_with_delta['positive_delta'] = positive_delta_list
        
        # Filter out the last session (no next step)
        linucb_with_delta = linucb_with_delta[linucb_with_delta['session_idx'] < len(self.data) - 1]
        
        # Calculate metrics for each policy
        policies = {
            'LinUCB': linucb_with_delta,
            'Weakest-Skill-First': weakest_results,
            'Random': random_results
        }
        
        comparison = []
        
        for policy_name, results_df in policies.items():
            metrics = {
                'Policy': policy_name,
                'Avg Reward': results_df['reward'].mean(),
                'Std Reward': results_df['reward'].std(),
                'Avg Δoverall': results_df['delta_overall'].mean(),
                '% Positive Δoverall': (results_df['positive_delta'].mean() * 100),
                'Total Positive Steps': results_df['positive_delta'].sum(),
                'Total Negative Steps': (~results_df['positive_delta']).sum()
            }
            comparison.append(metrics)
        
        comparison_df = pd.DataFrame(comparison)
        
        # Store for later analysis
        self.policy_comparison = comparison_df
        self.baseline_results = {
            'linucb': linucb_with_delta,
            'weakest': weakest_results,
            'random': random_results
        }
        
        return comparison_df

## 5. Run Policy Comparison

In [ ]:
# Initialize policy evaluator
policy_eval = PolicyEvaluator(full_df, policy_df)

# Compare policies
policy_comparison = policy_eval.compare_policies()

print("\n" + "="*80)
print("POLICY PERFORMANCE COMPARISON")
print("="*80)
print(policy_comparison.to_string(index=False))

# Calculate improvements over baseline
linucb_reward = policy_comparison.iloc[0]['Avg Reward']
baseline_reward = policy_comparison.iloc[1]['Avg Reward']
random_reward = policy_comparison.iloc[2]['Avg Reward']

reward_vs_baseline = ((linucb_reward - baseline_reward) / abs(baseline_reward)) * 100 if baseline_reward != 0 else float('inf')
reward_vs_random = ((linucb_reward - random_reward) / abs(random_reward)) * 100 if random_reward != 0 else float('inf')

print(f"\n📈 LinUCB Performance:")
print(f"   vs Weakest-Skill-First: {reward_vs_baseline:+.1f}% reward")
print(f"   vs Random: {reward_vs_random:+.1f}% reward")

## 6. Visualize Policy Comparison

In [ ]:
# Reward distribution comparison
fig = go.Figure()

policies_data = {
    'LinUCB': policy_eval.baseline_results['linucb']['reward'],
    'Weakest-Skill-First': policy_eval.baseline_results['weakest']['reward'],
    'Random': policy_eval.baseline_results['random']['reward']
}

for policy_name, rewards in policies_data.items():
    fig.add_trace(go.Box(
        y=rewards,
        name=policy_name,
        boxmean='sd'
    ))

fig.update_layout(
    title='Reward Distribution by Policy',
    yaxis_title='Reward',
    height=500
)
fig.show()

# Cumulative reward over time
fig = go.Figure()

for policy_name, rewards in policies_data.items():
    cumulative_reward = np.cumsum(rewards)
    fig.add_trace(go.Scatter(
        x=list(range(len(cumulative_reward))),
        y=cumulative_reward,
        mode='lines+markers',
        name=policy_name
    ))

fig.update_layout(
    title='Cumulative Reward Over Sessions',
    xaxis_title='Session',
    yaxis_title='Cumulative Reward',
    height=500
)
fig.show()

## 7. Alignment Sanity Check

In [ ]:
def check_alignment(session_idx: int, data_df: pd.DataFrame, 
                   policy_df: pd.DataFrame, llm_features: List[str]) -> Dict:
    """
    Check if policy focus aligns with actual weaknesses.
    
    Returns alignment analysis for a specific session.
    """
    current_row = data_df.iloc[session_idx]
    policy_row = policy_df[policy_df['session_idx'] == session_idx].iloc[0]
    
    # Get rubric scores
    rubric_to_action = {
        'clarity and enthusiasm in pitch': 'clarity',
        'active listening and objection handling': 'active_listening',
        'effective call to action': 'call_to_action',
        'friendliness and respectful tone': 'friendliness'
    }
    
    rubric_scores = {
        action: current_row[rubric]
        for rubric, action in rubric_to_action.items()
    }
    
    # Find weakest rubric skill
    weakest_rubric_skill = min(rubric_scores, key=rubric_scores.get)
    weakest_rubric_score = rubric_scores[weakest_rubric_skill]
    
    # Get LLM feature scores
    llm_scores = {
        feat: current_row[feat]
        for feat in llm_features
    }
    
    # Map LLM features to skills (approximate mapping)
    llm_to_skill = {
        'enthusiasm': 'clarity',
        'cta_clarity': 'call_to_action',
        'objection_handling': 'active_listening',
        'question_ratio': 'active_listening',
        'empathy_score': 'friendliness',
        'collaborative_score': 'friendliness',
        'social_proof': 'friendliness'
    }
    
    # Find LLM-flagged weaknesses (bottom 33rd percentile)
    llm_threshold = 0.33
    llm_weaknesses = {
        llm_to_skill[feat]: score
        for feat, score in llm_scores.items()
        if score < llm_threshold
    }
    
    # Get policy decision
    selected_action = policy_row['selected_action']
    
    # Check alignment
    alignment = {
        'session_idx': session_idx,
        'selected_action': selected_action,
        'weakest_rubric_skill': weakest_rubric_skill,
        'weakest_rubric_score': weakest_rubric_score,
        'matches_weakest_rubric': selected_action == weakest_rubric_skill,
        'llm_weaknesses': llm_weaknesses,
        'matches_llm_weakness': selected_action in llm_weaknesses,
        'rubric_scores': rubric_scores,
        'llm_scores': llm_scores
    }
    
    return alignment

# Check alignment for first 3 sessions
print("\n" + "="*80)
print("ALIGNMENT SANITY CHECK")
print("="*80)

llm_features = ['objection_handling', 'question_ratio', 'cta_clarity',
                'empathy_score', 'collaborative_score', 'social_proof', 
                'enthusiasm']

for session_idx in range(min(3, len(policy_df))):
    alignment = check_alignment(session_idx, full_df, policy_df, llm_features)
    
    print(f"\n📍 Session {session_idx + 1}:")
    print(f"   Selected Action: {alignment['selected_action']}")
    print(f"   Weakest Rubric Skill: {alignment['weakest_rubric_skill']} (score: {alignment['weakest_rubric_score']:.1f})")
    print(f"   Matches Weakest Rubric: {'✓ YES' if alignment['matches_weakest_rubric'] else '✗ NO'}")
    print(f"   LLM-Flagged Weaknesses: {list(alignment['llm_weaknesses'].keys())}")
    print(f"   Matches LLM Weakness: {'✓ YES' if alignment['matches_llm_weakness'] else '✗ NO'}")
    
    # Show rubric scores for context
    print(f"\n   Rubric Scores:")
    for skill, score in sorted(alignment['rubric_scores'].items(), key=lambda x: x[1]):
        marker = "👉" if skill == alignment['selected_action'] else "  "
        print(f"   {marker} {skill}: {score:.1f}")

# Calculate overall alignment statistics
alignment_stats = []
for session_idx in range(len(policy_df)):
    if session_idx < len(full_df):
        alignment = check_alignment(session_idx, full_df, policy_df, llm_features)
        alignment_stats.append({
            'matches_weakest_rubric': alignment['matches_weakest_rubric'],
            'matches_llm_weakness': alignment['matches_llm_weakness']
        })

alignment_df = pd.DataFrame(alignment_stats)

print(f"\n📊 Overall Alignment Statistics:")
print(f"   Matches Weakest Rubric: {alignment_df['matches_weakest_rubric'].mean():.1%}")
print(f"   Matches LLM Weakness: {alignment_df['matches_llm_weakness'].mean():.1%}")

## 8. Export Evaluation Results

In [ ]:
import os

# Create evaluation directory
os.makedirs('data/evaluation_results', exist_ok=True)

# Save ablation results
ablation_results.to_csv('data/evaluation_results/feature_ablation.csv', index=False)

# Save policy comparison
policy_comparison.to_csv('data/evaluation_results/policy_comparison.csv', index=False)

# Save alignment statistics
alignment_df.to_csv('data/evaluation_results/alignment_statistics.csv', index=False)

# Create comprehensive evaluation report
evaluation_report = {
    'feature_usefulness': {
        'ablation_table': ablation_results.to_dict('records'),
        'summary': {
            'llm_feature_r2_improvement_pct': r2_improvement,
            'llm_feature_auc_improvement_pct': auc_improvement,
            'best_r2': ablation_results['R² (Regression)'].max(),
            'best_auc': ablation_results['AUC (Classification)'].max()
        }
    },
    'policy_performance': {
        'comparison_table': policy_comparison.to_dict('records'),
        'summary': {
            'linucb_avg_reward': linucb_reward,
            'linucb_vs_baseline_pct': reward_vs_baseline,
            'linucb_vs_random_pct': reward_vs_random,
            'linucb_positive_delta_pct': policy_comparison.iloc[0]['% Positive Δoverall']
        }
    },
    'alignment_sanity': {
        'matches_weakest_rubric_pct': float(alignment_df['matches_weakest_rubric'].mean()),
        'matches_llm_weakness_pct': float(alignment_df['matches_llm_weakness'].mean()),
        'total_sessions_analyzed': len(alignment_df)
    }
}

with open('data/evaluation_results/evaluation_report.json', 'w') as f:
    json.dump(evaluation_report, f, indent=2)

print("\n✅ Evaluation results saved to 'data/evaluation_results/'")
print("   - feature_ablation.csv")
print("   - policy_comparison.csv")
print("   - alignment_statistics.csv")
print("   - evaluation_report.json")